In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Data Credits : Obtained from Keggle TMBD Movie Data set 

Will be working on the data to build an Movie Recommender System
Steps:
1. Pre-Processing
2. Vectorization of Data
3. Build a recommender function

In [ ]:
# Load the data
movies = pd.read_csv('/Users/dhanrajtentu/Downloads/Machine Learning Projects/Movie Recomender System/tmdb_5000_movies.csv')
credits = pd.read_csv("/Users/dhanrajtentu/Downloads/Machine Learning Projects/Movie Recomender System/tmdb_5000_credits.csv")

In [ ]:
movies.head()

In [ ]:
(credits.head())

In [ ]:
movies.shape

In [ ]:
credits.shape

1. Preprocessing

In [ ]:
# merge both DataFrames
movies = movies.merge(credits, on='title')
movies.merge(credits, on='title').shape

In [ ]:
movies.head(3)

In [ ]:
movies.columns

In [ ]:
movies.iloc[4805].keywords

In [ ]:
movies.iloc[0].overview

In [ ]:
movies.iloc[0].cast

In [ ]:
#genres
#keywords
#orginal_title
#overview
#movie_id
#cast
#crew

In [ ]:
# Check and, Remove non necessary columns
movies = movies[['title','movie_id','overview','keywords','genres','cast','crew']]

In [ ]:
movies.head()

In [ ]:
movies.isna().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
import ast
#help(ast)

In [ ]:
# Extract Main keywords from keywords column

def get_keywords(obj):
    list_of_words = []
    for i in ast.literal_eval(obj):
        list_of_words.append(i['name'])
    return list_of_words    

In [ ]:
movies['keywords'] = movies['keywords'].apply(get_keywords)

In [ ]:
movies['genres'] = movies['genres'].apply(get_keywords)

In [ ]:
# get top three cast (actors ) from cast column
movies['cast'] = movies['cast'].apply(get_keywords)

In [ ]:
movies['cast'] = movies['cast'].apply(lambda a : a[0:3])
movies.head()

In [ ]:
movies.iloc[0].crew

In [ ]:
# get the name of the director
def fetch_director(obj):
    list_of_words = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            list_of_words.append(i['name'])
    return list_of_words        
        

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)
movies.sample(5)

In [ ]:
def collapse(stri):
    l1 = []
    for i in stri:
        l1.append(i.replace(' ',''))
    return l1    

In [ ]:
movies['keywords'] = movies['keywords'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies.head()

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [ ]:
# Merge the Information about the movie to tags
movies['tags']= movies['overview'] + movies['keywords'] + movies['genres'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [ ]:
# create new df with tags column
new_df = movies[['title','movie_id','tags']]
new_df['tags'] = new_df['tags'].apply(lambda x: ' '.join(x))
new_df.head()


In [ ]:
new_df['tags']= new_df['tags'].apply(lambda x: x.lower())

2. Vectorization of the columns

In [ ]:
# Convert the tags in vector form

# import libriaries
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features = 5000, stop_words='english')

vectors = cv.fit_transform((new_df['tags'])).toarray() # The words and strings have been sucessfully converted to vector points

vectors.shape

In [ ]:
# find the distance of one array wiht all the array as the distance tells us which one is similar to which
# will find cosine distance instead of eqadian as features are 5k.

from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vectors)

In [ ]:
similarity

3. Build a system

In [ ]:
# Build a funtion to recommend new movies

def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),key=lambda x:x[1],reverse=True)
    print("Movies recommended for you:\n")
    for i in distances[1:6]:
        
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend("Pirates of the Caribbean: At World's End")

In [ ]:
recommend('Gandhi')